# Use Our Classifier for Inference

In [1]:
import torch

import loader
import models
import utility

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")

Using device: cuda


In [ ]:
def philosophize_this(inference_df, model, device, label_to_category_mapping, print_info=False):

    def logits_to_prediction(logits, label_to_category_mapping):
        sorted_indices = torch.argsort(logits, descending=True)    
        predictions = [(label_to_category_mapping[str(idx.item())], logits[idx].item()) for idx in sorted_indices]
        return predictions

    full_logits = torch.zeros((len(inference_df), len(label_to_category_mapping)))
    for i, row in inference_df.iterrows():
        model.eval()
        with torch.no_grad():
            logits = model(row['embedding'].to(device)).cpu()
            logits = torch.nn.functional.softmax(logits, dim=-1)
        full_logits[i] = logits
        predictions = logits_to_prediction(logits, label_to_category_mapping)
        
        if print_info:
            print(f"{[f'{c}, {v:.2f}' for c, v in predictions]}")
            print(row['chunk_text'])
            print(f"{'='*100}\n")

    final_prediction = logits_to_prediction(full_logits.mean(dim=0), label_to_category_mapping)
    print(f"Final philosophical prediction of your input text:\n{[f'{c}, {v:.2f}' for c, v in final_prediction]}")

In [ ]:
filename_category_mapping, category_label_mapping = loader.load_labeling_mappings()
filename_label_mapping = filename_category_mapping
for key, value in filename_label_mapping.items():
    filename_label_mapping[key] = int(category_label_mapping[value])

label_to_category_mapping = {v: k for k, v in category_label_mapping.items()}

# Load the model
filename = 'classifier_29k_11242024_191324.pth'
model = models.Classifier.load_model(filename).to(device)

Model loaded from model_weights\classifier_29k_11242024_191324.pth


c:\Users\lucas\Desktop\UCSD\256\philosophical_oracle\models\classifier.py:50: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  model_info = torch.load(filepath)


In [6]:
text_file = 'data/inference/rtwbackpack.txt'
with open(text_file, 'r', encoding='utf-8') as file:
    inference_text = file.read()


inference_df = loader.embed_texts(inference_text, chunk_size=2000, chunk_overlap=50, print_info=False)
philosophize_this(inference_df, model, device, label_to_category_mapping, print_info=True)
print(f"{'='*100}\n")

['transcendentalism, 0.44', 'absurdism, 0.38', 'epicureanism, 0.15', 'existentialism, 0.02', 'stoicism, 0.01', 'buddhism, 0.00', 'rationalism, 0.00']
This is all I took with me:

4 shorts, 2 pants, 2 swimsuits
7 shirts
1 jacket
6 pairs of underwear
7 pairs of socks
One backpack towel, hat, pair of sunglasses
3 pairs of shoes
Water bottle, day pack, dry sack, water bladder
Passport, headphones, laptop, chargers, sketchbook (yellow bag), belt (blue bag)
Couple fun things for Croatia music festival in blue bag
Lost these Chacos when flying *(RIP)*, picked up a new shirt and shorts for Croatia at a thrift store in Barcelona, bought a pair of sunglasses from a friend’s store in Greece, and got a shirt in Byron Bay at the end of the trip. Otherwise it stayed all the same.
If it wasn’t for Croatia and some of the long hikes I wanted to do, I would’ve packed less.

---

I’ve grown up with abundance my entire life, so I wanted to see if I could comfortably live off of bare necessities. I found 